# 🚀 S1W3D4 (Day 17): 编写评估循环

**今日重点**：模式切换、计算图阻断、预测逻辑

## 🎯 核心原理 (一定要弄清楚！)

在写代码前，必须彻底理解这三个概念，它们是面试中关于评估过程的必考题。

### 🟢 原理一：`model.eval()` vs `model.train()`

  * **问题**：为什么考试时要喊一声“现在是考试模式”？
  * **原理**：神经网络中有些层在“训练”和“预测”时的表现是**完全不同**的。
      * **Dropout**：训练时随机“关掉”一部分神经元（防止依赖），预测时必须**全开**（火力全开）。
      * **BatchNorm**：训练时用当前 Batch 的均值方差做归一化；预测时必须用**历史累计**的全局均值方差（保证稳定性）。
  * **结论**：如果你忘了写 `model.eval()`，你的准确率可能会莫名其妙地低，或者每次运行结果都不一样。

### 🟢 原理二：`torch.no_grad()` (计算图阻断)

  * **问题**：为什么评估时要加这句代码？
  * **原理**：
      * PyTorch 默认会记录所有操作构建“计算图”以便反向传播求导。这非常消耗显存和算力。
      * **考试不需要求导**！我们只需要结果，不需要更新参数。
      * `with torch.no_grad():` 就像一个开关，告诉 PyTorch：“在这个缩进块里，别浪费内存去记梯度了，我不需要反向传播。”
  * **结论**：不加也能跑，但会**浪费巨量显存**，甚至导致 Out of Memory (OOM)，且速度变慢。

### 🟢 原理三：Logits 到 预测结果 (`argmax`)

  * **问题**：模型输出的是 `[-1.2, 2.5, 0.1, ...]` 这一堆数，怎么变成数字 "1"？
  * **原理**：
      * 模型输出的叫 **Logits** (得分)。得分越高，概率越大。
      * 我们不需要用 Softmax 算出具体概率（比如 80%），我们只需要知道\*\*“哪个下标的得分最高”\*\*。
      * **`torch.argmax(output, dim=1)`**：这个函数会返回最大值的**索引 (Index)**。
      * 例如：`[0.1, 0.9, 0.0]` -\> 最大值是 0.9 -\> 它的索引是 **1** -\> 预测结果就是数字 1。

## 💻 准备工作：环境与导入

首先需要设置好`src`的路径。

In [1]:
import sys
import os
import torch

# --- 1. 挂载项目根目录 ---
project_root = os.path.abspath(os.path.join(os.getcwd(), "../..")) 
if project_root not in sys.path:
    sys.path.append(project_root)

# --- 2. 导入 src 模块 ---
from src.utils import get_data_loaders
from src.models import SimpleCNN
from src.engine import train, evaluate  # 导入引擎

# --- 3. 准备设备 ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# --- 4. 加载数据与模型 ---
# 这里的 test_loader 是今天的重点！
train_loader, test_loader = get_data_loaders(batch_size=64, data_root=os.path.join(project_root, 'data'))

# 实例化模型
model = SimpleCNN().to(device)

# 此时模型是随机初始化的（很笨），预测准确率应该在 10% 左右
print("✅ 根目录准备就绪")

✅ 根目录准备就绪


## 📝 编写评估函数 (Evaluation Function)

已写入`../../src/models/engine.py`

In [ ]:
# import torch.nn as nn

# def evaluate(model, device, test_loader):
#     # 1. 切换模式 (原理一)
#     model.eval()
    
#     # 定义统计变量
#     test_loss = 0
#     correct = 0
    
#     # 定义损失函数 (用于计算测试集的平均 Loss)
#     criterion = nn.CrossEntropyLoss(reduction='sum') 
#     # 注意：这里用了 reduction='sum'，表示把一个 Batch 的 Loss 加起来，而不是求平均。
#     # 这样方便最后除以总样本数。
    
#     # 2. 阻断计算图 (原理二)
#     with torch.no_grad():
#         # 遍历测试集
#         for data, target in test_loader:
#             data, target = data.to(device), target.to(device)
            
#             # 前向传播
#             output = model(data)
            
#             # 计算 Batch 总 Loss
#             test_loss += criterion(output, target).item()
            
#             # 3. 获取预测结果 (原理三)
#             # output 形状: [64, 10]
#             # dim=1 表示在“列”方向（10个类别）找最大值
#             pred = output.argmax(dim=1, keepdim=True) 
            
#             # 计算正确个数
#             # pred.eq(target) 会对比预测值和真实值，相等的为 True(1)，不等的为 False(0)
#             correct += pred.eq(target.view_as(pred)).sum().item()

#     # 计算平均 Loss 和 准确率
#     test_loss /= len(test_loader.dataset) # 平均 Loss = 总 Loss / 总样本数
#     accuracy = 100. * correct / len(test_loader.dataset) # 准确率 %

#     print(f'\n🔴 [Test Set] Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} ({accuracy:.2f}%)\n')
    
#     return accuracy

## 🧪 4. 运行测试 (Baseline)

现在我们的模型还没训练，是一张白纸。我们先测一下，看看它是不是只有 10% 的准确率（瞎猜）。

In [4]:
# 这里的 model 是刚初始化的
acc = evaluate(model, device, test_loader)


🔴 [Test Set] Average loss: 2.3056, Accuracy: 740/10000 (7.40%)



## 🔄 5. 终极融合：训练 + 评估 (Train & Eval Loop)

现在我们将昨天的 **Train** 和今天的 **Evaluate** 结合起来。
**这是深度学习脚本的完全体形态。**

我们希望：**每训练完一个 Epoch，就立刻在测试集上考一次试，看看有没有进步。**

In [2]:
import torch.optim as optim

# --- 主循环 ---
# 重新初始化模型和优化器
model = SimpleCNN().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)

EPOCHS = 3

for epoch in range(1, EPOCHS + 1):
    # 1. 上课 (训练)
    train(model, device, train_loader, optimizer, epoch, log_interval = 50)
    
    # 2. 考试 (评估)
    acc = evaluate(model, device, test_loader)
    
    # 这里可以加一个“保存模型”的逻辑：如果 acc 是历史最高的，就 save 下来

Train Epoch: 1 [50/938 (5%)]	Loss: 0.531822
Train Epoch: 1 [100/938 (11%)]	Loss: 0.330147
Train Epoch: 1 [150/938 (16%)]	Loss: 0.116773
Train Epoch: 1 [200/938 (21%)]	Loss: 0.270419
Train Epoch: 1 [250/938 (27%)]	Loss: 0.219862
Train Epoch: 1 [300/938 (32%)]	Loss: 0.091976
Train Epoch: 1 [350/938 (37%)]	Loss: 0.236950
Train Epoch: 1 [400/938 (43%)]	Loss: 0.054950
Train Epoch: 1 [450/938 (48%)]	Loss: 0.041809
Train Epoch: 1 [500/938 (53%)]	Loss: 0.065647
Train Epoch: 1 [550/938 (59%)]	Loss: 0.211401
Train Epoch: 1 [600/938 (64%)]	Loss: 0.068390
Train Epoch: 1 [650/938 (69%)]	Loss: 0.011472
Train Epoch: 1 [700/938 (75%)]	Loss: 0.080514
Train Epoch: 1 [750/938 (80%)]	Loss: 0.033956
Train Epoch: 1 [800/938 (85%)]	Loss: 0.054457
Train Epoch: 1 [850/938 (91%)]	Loss: 0.074679
Train Epoch: 1 [900/938 (96%)]	Loss: 0.084595

🔴 [Test set] Average loss: 0.0668, Accuracy: 9787/10000 (97.87%)

Train Epoch: 2 [50/938 (5%)]	Loss: 0.117493
Train Epoch: 2 [100/938 (11%)]	Loss: 0.052727
Train Epoch: 2 [1

应该会看到：
> Epoch 1 结束 -\> Accuracy 从 10% 飙升到 95% 以上。
> Epoch 2 结束 -\> Accuracy 继续微涨，可能到 98%。
> Epoch 3 结束 -\> 收敛。

## 🧠 今日复盘 (Self-Check)

请对着代码回答以下问题，确认你是否真的“弄清楚了原理”：

1.  **`argmax(dim=1)` 是什么意思？**
      * 它是在找“概率最大”的那个类别的**索引**。比如 `[0.1, 0.8, 0.1]`，最大值 0.8 在索引 1，所以预测结果是 1。
2.  **如果在 `evaluate` 函数里删掉 `with torch.no_grad():` 会怎样？**
      * 代码也能跑，但是会消耗大量显存（因为记录了梯度），而且运行速度会变慢。
3.  **为什么要除以 `len(test_loader.dataset)`？**
      * 因为我们在算 Loss 时用了 `sum`（累加），为了得到“平均每个样本的 Loss”，必须除以样本总数。

如果这三个问题都难不倒你，那么 S1W3D4 任务完成！你的模型现在既能学，又能考，已经是一个成熟的 AI 了！🤖